# Request ERA5 Monthly Variables

## Monthly ERA5 variables

Classic download 
## 'Essential variables'

There is a new product for Monthly data, the 'essential climate variables'. This is very cool BUT it seems to only be at the high spatial resolution, which means re-working the population data! Therefore still download the classic monthly data

In [1]:
from pathlib import Path

import cdsapi
import joblib
import dask
from dask.distributed import Client

In [2]:
MAX_YEAR = 2022

In [3]:
era5_data_folder = Path('~/Shared/Data/weather/era5_0.25deg').expanduser()

monthly_data_folder = era5_data_folder / 'monthly_means'

essential_vars_folder = era5_data_folder / 'essential_monthly_variables'
# essential_vars_folder.mkdir(exist_ok=True, parents=True)

In [4]:
assert monthly_data_folder.is_dir()
assert essential_vars_folder.is_dir()

In [5]:
def download_cds_request(download_path, cds_dataset_key, cds_options, overwrite=False):
    
    if Path(download_path).is_file() and overwrite is False:
        # Do nothing if the file already exists
        return f'Skipped {download_path}'
    
    if isinstance(download_path, Path):
        download_path = str(download_path)
        
        
    c = cdsapi.Client()
    c.retrieve(
        cds_dataset_key,
        cds_options,
        download_path)
    return f'Downloaded {download_path}'

In [6]:
YEARS = list(range(1979, MAX_YEAR+1)) # max desired year + 1

# Download Monthly dataset

In [7]:
# if not specified, gets the 0.25 grid
# GRID = "0.5/0.5"

In [7]:
YEARS[-1:]

[2022]

In [8]:
 year = 2021
(monthly_data_folder / f'{year}_monthly_mean.grib').exists()

True

In [9]:
joblib.Parallel(n_jobs=10, verbose=10)(
    joblib.delayed(download_cds_request)(
        monthly_data_folder / f'{year}_monthly_mean.grib',
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'grib',
#             "grid": GRID,
            'variable': [
                '2m_dewpoint_temperature', '2m_temperature', 'surface_pressure',
            ],
            'product_type': 'monthly_averaged_reanalysis',
            'year': f'{year}',
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'time': '00:00',
        })
        for year in YEARS if not (monthly_data_folder / f'{year}_monthly_mean.grib').exists())

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
2023-02-01 14:38:16,447 INFO Welcome to the CDS
2023-02-01 14:38:16,447 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-02-01 14:38:16,583 INFO Request is queued
2023-02-01 14:38:21,407 INFO Request is running
2023-02-01 14:38:29,901 INFO Request is completed
2023-02-01 14:38:29,901 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data1/adaptor.mars.internal-1675258699.5923347-20597-12-6eef1b52-7153-4f63-a5b2-81006aba2671.grib to /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/monthly_means/2022_monthly_mean.grib (71.3M)
2023-02-01 14:38:34,342 INFO Download rate 16.1M/s  
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:   18.4s
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   18.4s finished


['Downloaded /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/monthly_means/2022_monthly_mean.grib']

# Download Essential Climate Variables dataset

In [10]:
# Note: not necessary to use distributed
# with joblib.parallel_backend('dask'):

joblib.Parallel(n_jobs=10, verbose=10)(
        joblib.delayed(download_cds_request)(
            essential_vars_folder / f'variables_{year}.zip',
            'ecv-for-climate-change',
            {
                'variable': [
                'precipitation', 'surface_air_temperature',
            ],
            'product_type': 'monthly_mean',
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'origin': 'era5',
            'format': 'zip',
            'time_aggregation': '1_month_mean',
            'year': f'{year}',
        })
        for year in YEARS if not (essential_vars_folder / f'variables_{year}.zip').exists())
    

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
2023-02-01 14:39:41,930 INFO Welcome to the CDS
2023-02-01 14:39:41,930 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/ecv-for-climate-change
2023-02-01 14:39:42,009 INFO Request is queued
2023-02-01 14:39:43,043 INFO Request is running
2023-02-01 14:40:31,513 INFO Request is completed
2023-02-01 14:40:31,513 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data6/dataset-ecv-for-climate-change-09369dd4-6d0c-4907-a806-efaabc9f14eb.zip to /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/essential_monthly_variables/variables_2022.zip (36M)
2023-02-01 14:40:33,286 INFO Download rate 20.3M/s  
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:   51.5s
[Parallel(n_jobs=10)]: Done   1 out of   1 | elapsed:   51.6s finished


['Downloaded /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/essential_monthly_variables/variables_2022.zip']